Analyzing noncoding variation associated with disease is a major application of Basenji. I now offer several tools to enable that analysis. If you have a small set of variants and know what datasets are most relevant, [basenji_sat_vcf.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat_vcf.py) lets you perform a saturation mutagenesis of the variant and surrounding region to see the relevant nearby motifs.

If you want scores measuring the influence of those variants on all datasets,
 * [basenji_sad.py](https://github.com/calico/basenji/blob/master/bin/basenji_sad.py) computes my SNP activity difference (SAD) score--the predicted change in aligned fragments to the region.
 * [basenji_sed.py](https://github.com/calico/basenji/blob/master/bin/basenji_sed.py) computes my SNP expression difference (SED) score--the predicted change in aligned fragments to gene TSS's.

Here, I'll demonstrate those two programs. You'll need
 * Trained model
 * Input file (FASTA or HDF5 with test_in/test_out)

First, you can either train your own model in the [Train/test tutorial](https://github.com/calico/basenji/blob/master/tutorials/train_test.ipynb) or download one that I pre-trained.

In [ ]:
if not os.path.isfile('models/gm12878_d10.tf.meta'):
    subprocess.call('curl -o models/gm12878_d10.tf.index https://storage.googleapis.com/basenji_tutorial_data/model_gm12878_d10.tf.index', shell=True)
    subprocess.call('curl -o models/gm12878_d10.tf.meta https://storage.googleapis.com/basenji_tutorial_data/model_gm12878_d10.tf.meta', shell=True)
    subprocess.call('curl -o models/gm12878_d10.tf.data-00000-of-00001 https://storage.googleapis.com/basenji_tutorial_data/model_gm12878_d10.tf.data-00000-of-00001', shell=True)

We'll bash the PIM1 promoter to see what motifs drive its expression. I placed a 262 kb FASTA file surrounding the PIM1 TSS in data/pim1.fa, so we'll use [basenji_sat.py](https://github.com/calico/basenji/blob/master/bin/basenji_sat.py).

The most relevant options are:

| Option/Argument | Value | Note |
|:---|:---|:---|
| -g | data/human.hg19.genome | Genome assembly chromosome length to bound gene sequences. |
| -f | 20 | Figure width, that I usually scale to 10x the saturation mutageneis region |
| -l | 200 | Saturation mutagenesis region in the center of the given sequence(s) |
| -o | pim1_sat | Outplot plot directory. |
| -t | 0,38 | Target indexes. 0 is a DNase and 38 is CAGE, as you can see in data/gm12878_wigs.txt. |
| params_file | models/params_small_sat.txt | Table of parameters to setup the model architecture and optimization parameters. |
| model_file | models/gm12878_d10.tf | Trained saved model prefix. |
| input_file | data/pim1.fa | Either FASTA or HDF5 with test_in/test_out keys. |

In [ ]:
! basenji_sat.py -f 20 -l 200 -o pim1_sat -t 0,38 models/params_small_sat.txt models/gm12878_d10.tf data/pim1.fa

{'target_pool': 128, 'cnn_filter_sizes': [22, 1, 6, 6, 6, 3], 'cnn_filters': [128, 128, 160, 200, 250, 256], 'dcnn_filters': [32, 32, 32, 32, 32, 32], 'adam_beta2': 0.98, 'num_targets': 39, 'loss': 'poisson', 'batch_size': 1, 'learning_rate': 0.002, 'cnn_dropout': 0.05, 'adam_beta1': 0.97, 'dense': 1, 'cnn_pool': [1, 2, 4, 4, 4, 1], 'full_dropout': 0.05, 'link': 'softplus', 'full_units': 384, 'batch_buffer': 16384, 'batch_renorm': 1, 'dcnn_filter_sizes': [3, 3, 3, 3, 3, 3], 'dcnn_dropout': 0.1}
Targets pooled by 128 to length 2048
Convolution w/ 128 4x22 filters strided by 1
Batch normalization
ReLU
Dropout w/ probability 0.050
Convolution w/ 128 128x1 filters strided by 1
Batch normalization
ReLU
Max pool 2
Dropout w/ probability 0.050
Convolution w/ 160 128x6 filters strided by 1
Batch normalization
ReLU
Max pool 4
Dropout w/ probability 0.050
Convolution w/ 200 160x6 filters strided by 1
Batch normalization
ReLU
Max pool 4
Dropout w/ probability 0.050
Convolution w/ 250 200x6 filter

The saturated mutagenesis heatmaps go into pim1_sat

In [ ]:
from IPython.display import IFrame
IFrame('pim1_sat/???.pdf', width=1200, height=400)

Describe the output...